<a href="https://colab.research.google.com/github/AIM-Harvard/aimi_alpha/blob/main/aimi/platipy/notebooks/platipy_mwe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ModelHub - Cardiac Substructures Segmentation**

This notebook provides an example of how to run an end-to-end (cloud-based) data analysis using the PlatiPy hybrid cardiac substructures segmentation pipeline.

The way all the operations are executed - from pulling data to data postprocessing and the standardisation of the results - have the goal of promoting transparency and reproducibility.

## **Environment Setup**

This demo notebook is intended to be run using a GPU.

To access a free GPU on Colab:
`Edit > Notebooks Settings`.

From the dropdown menu under `Hardware accelerator`, select `GPU`. Let's check the Colab instance is indeed equipped with a GPU.

In [ ]:
import os
import sys
import shutil

import yaml

import time
import tqdm


# useful information
curr_dir = !pwd
curr_droid = !hostname
curr_pilot = !whoami

print(time.asctime(time.localtime()))

print("\nCurrent directory :", curr_dir[-1])
print("Hostname          :", curr_droid[-1])
print("Username          :", curr_pilot[-1])

print("Python version    :", sys.version.split('\n')[0])

Thu Sep 15 14:50:39 2022

Current directory : /content
Hostname          : 8ea850de62b8
Username          : root
Python version    : 3.7.13 (default, Apr 24 2022, 01:04:09) 


The authentication to Google is necessary to run BigQuery queries.

Every operation throughout the whole notebook (BigQuery, fetching data from the IDC buckets) is completely free. The only thing that is needed in order to run the notebook is the set-up of a Google Cloud project. In order for the notebook to work as intended, you will need to specify the name of the project in the cell after the authentication one.

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import files
from google.cloud import storage
from google.cloud import bigquery as bq

# INSERT THE NAME OF YOUR PROJECT HERE!
project_name = "idc-sandbox-000"
#project_name = "sage-buttress-323909"

Throughout this Colab notebook, for image pre-processing we will use [Plastimatch](https://plastimatch.org), a reliable and open source software for image computation. We will be running Plastimatch using the simple [PyPlastimatch](https://github.com/AIM-Harvard/pyplastimatch/tree/main/pyplastimatch) python wrapper. 

In [ ]:
%%capture
!apt install plastimatch

In [ ]:
# check plastimatch was correctly installed
!plastimatch --version

plastimatch version 1.7.0


---

Start by cloning the AIME hub repository on the Colab instance.

The AIME hub repository stores all the code we will use for pulling, preprocessing, processing, and postprocessing the data for this use case - as long as the others shared through AIME hub.

In [ ]:
!git clone https://github.com/AIM-Harvard/aimi_alpha.git aime

Cloning into 'aime'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 138 (delta 65), reused 94 (delta 32), pack-reused 0
Receiving objects: 100% (138/138), 769.42 KiB | 3.81 MiB/s, done.
Resolving deltas: 100% (65/65), done.


To organise the DICOM data in a more common (and human-understandable) fashion after downloading those from the buckets, we will make use of [DICOMSort](https://github.com/pieper/dicomsort). 

DICOMSort is an open source tool for custom sorting and renaming of dicom files based on their specific DICOM tags. In our case, we will exploit DICOMSort to organise the DICOM data by `PatientID` and `Modality` - so that the final directory will look like the following:

```
data/raw/nsclc-radiomics/dicom/$PatientID
 └─── CT
       ├─── $SOPInstanceUID_slice0.dcm
       ├─── $SOPInstanceUID_slice1.dcm
       ├───  ...
       │
      RTSTRUCT 
       ├─── $SOPInstanceUID_RTSTRUCT.dcm
      SEG
       └─── $SOPInstanceUID_RTSEG.dcm

```

In [ ]:
!git clone https://github.com/pieper/dicomsort dicomsort

Cloning into 'dicomsort'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 130 (delta 0), reused 1 (delta 0), pack-reused 126
Receiving objects: 100% (130/130), 44.12 KiB | 1.00 MiB/s, done.
Resolving deltas: 100% (63/63), done.


We will also use DCMQI for converting the resulting segmentation into standard DICOM SEG objects.

In [ ]:
%%capture
dcmqi_release_url = "https://github.com/QIICR/dcmqi/releases/download/v1.2.4/dcmqi-1.2.4-linux.tar.gz"
dcmqi_download_path = "/content/dcmqi-1.2.4-linux.tar.gz"
dcmqi_path = "/content/dcmqi-1.2.4-linux"

!wget -O $dcmqi_download_path $dcmqi_release_url

!tar -xvf $dcmqi_download_path

!mv $dcmqi_path/bin/* /bin

---

In [ ]:
%%capture
!pip install git+https://github.com/pyplati/platipy.git
!pip install pyplastimatch nnunet ipywidgets

In [ ]:
%%capture
!pip install matplotlib==2.2.3

In [ ]:
import shutil
import random

import json
import pprint
import numpy as np
import pandas as pd

import pydicom
import nibabel as nib
import SimpleITK as sitk
import pyplastimatch as pypla

print("Python version               : ", sys.version.split('\n')[0])
print("Numpy version                : ", np.__version__)

# ----------------------------------------

#everything that has to do with plotting goes here below
import matplotlib
matplotlib.use("agg")

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

%matplotlib inline
%config InlineBackend.figure_format = "png"

import ipywidgets as ipyw

## ----------------------------------------

# create new colormap appending the alpha channel to the selected one
# (so that we don't get a \"color overlay\" when plotting the segmask superimposed to the CT)
cmap = plt.cm.Reds
my_reds = cmap(np.arange(cmap.N))
my_reds[:, -1] = np.linspace(0, 1, cmap.N)
my_reds = ListedColormap(my_reds)

cmap = plt.cm.Greens
my_greens = cmap(np.arange(cmap.N))
my_greens[:, -1] = np.linspace(0, 1, cmap.N)
my_greens = ListedColormap(my_greens)

cmap = plt.cm.Blues
my_blues = cmap(np.arange(cmap.N))
my_blues[:, -1] = np.linspace(0, 1, cmap.N)
my_blues = ListedColormap(my_blues)

cmap = plt.cm.spring
my_spring = cmap(np.arange(cmap.N))
my_spring[:, -1] = np.linspace(0, 1, cmap.N)
my_spring = ListedColormap(my_spring)
## ----------------------------------------

import seaborn as sns

Python version               :  3.7.13 (default, Apr 24 2022, 01:04:09) 
Numpy version                :  1.21.6


Provided everything was set up correctly, we can run the BigQuery query and get all the information we need to download the testing data from the IDC platform.

For this specific use case, we are going to be working with the NSCLC-Radiomics collection (Chest CT scans of lung cancer patients, with manual delineation of various organs at risk).

In [ ]:
%%bigquery --project=$project_name cohort_df

SELECT
  dicom_pivot_v11.PatientID,
  dicom_pivot_v11.collection_id,
  dicom_pivot_v11.source_DOI,
  dicom_pivot_v11.StudyInstanceUID,
  dicom_pivot_v11.SeriesInstanceUID,
  dicom_pivot_v11.SOPInstanceUID,
  dicom_pivot_v11.gcs_url
FROM
  `bigquery-public-data.idc_v11.dicom_pivot_v11` dicom_pivot_v11
WHERE
  StudyInstanceUID IN (
    SELECT
      StudyInstanceUID
    FROM
      `bigquery-public-data.idc_v11.dicom_pivot_v11` dicom_pivot_v11
    WHERE
      StudyInstanceUID IN (
        SELECT
          StudyInstanceUID
        FROM
          `bigquery-public-data.idc_v11.dicom_pivot_v11` dicom_pivot_v11
        WHERE
          (
            LOWER(
              dicom_pivot_v11.SegmentedPropertyTypeCodeSequence
            ) LIKE LOWER('80891009:SCT')
          )
        GROUP BY
          StudyInstanceUID
        INTERSECT DISTINCT
        SELECT
          StudyInstanceUID
        FROM
          `bigquery-public-data.idc_v11.dicom_pivot_v11` dicom_pivot_v11
        WHERE
          (
            dicom_pivot_v11.collection_id IN ('Community', 'nsclc_radiomics')
          )
        GROUP BY
          StudyInstanceUID
      )
    GROUP BY
      StudyInstanceUID
  )
GROUP BY
  dicom_pivot_v11.PatientID,
  dicom_pivot_v11.collection_id,
  dicom_pivot_v11.source_DOI,
  dicom_pivot_v11.StudyInstanceUID,
  dicom_pivot_v11.SeriesInstanceUID,
  dicom_pivot_v11.SOPInstanceUID,
  dicom_pivot_v11.gcs_url
ORDER BY
  dicom_pivot_v11.PatientID ASC,
  dicom_pivot_v11.collection_id ASC,
  dicom_pivot_v11.source_DOI ASC,
  dicom_pivot_v11.StudyInstanceUID ASC,
  dicom_pivot_v11.SeriesInstanceUID ASC,
  dicom_pivot_v11.SOPInstanceUID ASC,
  dicom_pivot_v11.gcs_url ASC

In [ ]:
# this works as intended only if the BQ query parses data from a single dataset
# if not, feel free to set the name manually!
dataset_name = cohort_df["collection_id"].values[0]

dataset_name

'nsclc_radiomics'

In [ ]:
# create the directory tree
!mkdir -p data models

!mkdir -p data/raw 
!mkdir -p data/raw/tmp data/raw/$dataset_name
!mkdir -p data/raw/$dataset_name/dicom

!mkdir -p data/processed
!mkdir -p data/processed/$dataset_name
!mkdir -p data/processed/$dataset_name/nii
!mkdir -p data/processed/$dataset_name/dicomseg

!mkdir -p data/model_input/
!mkdir -p data/platipy_output/

## **Parsing Cohort Information from BigQuery Tables**

We can check the various fields of the table we populated by running the BigQuery query.

This table will store one entry for each DICOM file in the dataset (therefore, expect thousands of rows!)

In [ ]:
pat_id_list = sorted(list(set(cohort_df["PatientID"].values)))

print("Total number of unique Patient IDs:", len(pat_id_list))

display(cohort_df.info())

display(cohort_df.head())

Total number of unique Patient IDs: 127
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15678 entries, 0 to 15677
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   PatientID          15678 non-null  object
 1   collection_id      15678 non-null  object
 2   source_DOI         15678 non-null  object
 3   StudyInstanceUID   15678 non-null  object
 4   SeriesInstanceUID  15678 non-null  object
 5   SOPInstanceUID     15678 non-null  object
 6   gcs_url            15678 non-null  object
dtypes: object(7)
memory usage: 857.5+ KB


None

,PatientID,collection_id,source_DOI,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,gcs_url
0,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.2.276.0.7230010.3.1.3.2323910823.11504.15972...,1.2.276.0.7230010.3.1.4.2323910823.11504.15972...,gs://idc-open-cr/eff917af-8a2a-42fe-9e12-22bce...
1,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.1004190115743500844746...,gs://idc-open-cr/f8cbf725-621d-4e18-8326-41789...
2,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.1031280376053401623619...,gs://idc-open-cr/c73b3d12-78b1-4456-9a88-91ba2...
3,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.1075071405629330534974...,gs://idc-open-cr/48b4ae0a-6936-44b4-a6bd-27c92...
4,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.1125363119759695902111...,gs://idc-open-cr/3c36a30a-630b-4183-b87d-8a238...


---

## **Set Run Parameters**

From this cell, we can configure the nnU-Net inference step - specifying, for instance, the type of model we want to run (among the four different models the framework provides), whether we want to use test time augmentation, or whether we want to export the soft probability maps of the segmentation masks.


In [ ]:
# FIXED PARAMETERS
data_base_path = "/content/data"
raw_base_path = "/content/data/raw/tmp"
sorted_base_path = os.path.join("/content/data/raw/", dataset_name, "dicom")

processed_base_path = os.path.join("/content/data/processed/", dataset_name)
processed_nifti_path = os.path.join(processed_base_path, "nii")

processed_dicomseg_path = os.path.join(processed_base_path, "dicomseg")

model_input_folder = "/content/data/model_input/"
model_output_folder = "/content/data/platipy_output/"

dicomseg_json_path = "/content/aime/aime/platipy/config/dicomseg_metadata.json"

## **Running the Analysis for a Single Patient**

In [ ]:
import aime.aime as aime

from aime import general_utils as aime_utils
from aime import platipy as aime_model

The following cells run all the processing pipeline, from pre-processing to post-processing.

In [ ]:
pat_id = random.choice(cohort_df["PatientID"].values)
pat_df = cohort_df[cohort_df["PatientID"] == pat_id].reset_index(drop = True)

display(pat_df.info())
display(pat_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   PatientID          97 non-null     object
 1   collection_id      97 non-null     object
 2   source_DOI         97 non-null     object
 3   StudyInstanceUID   97 non-null     object
 4   SeriesInstanceUID  97 non-null     object
 5   SOPInstanceUID     97 non-null     object
 6   gcs_url            97 non-null     object
dtypes: object(7)
memory usage: 5.4+ KB


None

,PatientID,collection_id,source_DOI,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,gcs_url
0,LUNG1-034,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2478224849816276412720...,1.2.276.0.7230010.3.1.3.2323910823.23408.15972...,1.2.276.0.7230010.3.1.4.2323910823.23408.15972...,gs://idc-open-cr/5e64b9ef-d40b-4fdc-8973-d361b...
1,LUNG1-034,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2478224849816276412720...,1.3.6.1.4.1.32722.99.99.2910936700174194202833...,1.3.6.1.4.1.32722.99.99.1401893232600672379868...,gs://idc-open-cr/0bc0d361-5b05-4db8-a90e-d6788...
2,LUNG1-034,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2478224849816276412720...,1.3.6.1.4.1.32722.99.99.3631014714041668019975...,1.3.6.1.4.1.32722.99.99.1045610775438006684042...,gs://idc-open-cr/8c3f76c7-da6b-47c8-b716-4d84d...
3,LUNG1-034,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2478224849816276412720...,1.3.6.1.4.1.32722.99.99.3631014714041668019975...,1.3.6.1.4.1.32722.99.99.1111115642964982084419...,gs://idc-open-cr/4741fc68-23a1-4015-b540-224e6...
4,LUNG1-034,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2478224849816276412720...,1.3.6.1.4.1.32722.99.99.3631014714041668019975...,1.3.6.1.4.1.32722.99.99.1174762546665195830791...,gs://idc-open-cr/1f3db776-23fc-45ab-97bc-c51d3...


In [ ]:
# init

print("Processing patient: %s"%(pat_id))

patient_df = cohort_df[cohort_df["PatientID"] == pat_id]

dicomseg_fn = pat_id + "_SEG.dcm"

input_nifti_fn = pat_id + ".nii.gz"
input_nifti_path = os.path.join(model_input_folder, input_nifti_fn)

pred_nifti_fn = pat_id + ".nii.gz"
pred_nifti_path = os.path.join(model_output_folder, pred_nifti_fn)

pred_softmax_folder_name = "pred_softmax"
pred_softmax_folder_path = os.path.join(processed_nifti_path, pat_id, pred_softmax_folder_name)

Processing patient: LUNG1-034


In [ ]:
# data cross-loading
aime_utils.gcs.download_patient_data(raw_base_path = raw_base_path,
                                     sorted_base_path = sorted_base_path,
                                     patient_df = patient_df,
                                     remove_raw = True)

Copying files from IDC buckets to /content/data/raw/tmp/LUNG1-034...
Done in 25.7324 seconds.

Sorting DICOM files...
Done in 1.04826 seconds.
Sorted DICOM data saved at: /content/data/raw/nsclc_radiomics/dicom/LUNG1-034
Removing un-sorted data at /content/data/raw/tmp/LUNG1-034...
... Done.


In [ ]:
# DICOM CT to NIfTI - required for the processing
aime_utils.preprocessing.pypla_dicom_ct_to_nifti(sorted_base_path = sorted_base_path,
                                                 processed_nifti_path = processed_nifti_path,
                                                 pat_id = pat_id, verbose = True)


Running 'plastimatch convert' with the specified arguments:
  --input /content/data/raw/nsclc_radiomics/dicom/LUNG1-034/CT
  --output-img /content/data/processed/nsclc_radiomics/nii/LUNG1-034/LUNG1-034_CT.nii.gz
... Done.


In [ ]:
# prepare the `model_input` folder for the inference phase
aime_utils.preprocessing.prep_input_data(processed_nifti_path = processed_nifti_path,
                                         model_input_folder = model_input_folder,
                                         pat_id = pat_id)

Copying /content/data/processed/nsclc_radiomics/nii/LUNG1-034/LUNG1-034_CT.nii.gz
to /content/data/model_input/LUNG1-034_0000.nii.gz...
... Done.


The following cell runs the hybrid cardiac segmentation.

Note that this operation can take up to 15 minutes:
*   The first step, dealing with whole heart segmentation via a custom nnU-Net model, runs on the GPU and is usually pretty fast (~2 minutes);
*   The second step, fitting the atlas to the target data, is CPU bound (and not really efficient nor multi-threaded) and it will take most of the time (~10 minutes);
*   To the above two, the user needs to add the time to download all the necessary models for the pipeline to run.



In [ ]:
aime_model.utils.processing.process_patient(pat_id = pat_id,
                                            model_input_folder = model_input_folder,
                                            model_output_folder = model_output_folder)

---

In [ ]:
processed_seg_folder = os.path.join(processed_nifti_path, pat_id, "platipy")
shutil.copytree(model_output_folder, processed_seg_folder)

'/content/data/processed/nsclc_radiomics/nii/LUNG1-034/platipy'

In [ ]:
aime_model.utils.postprocessing.nifti_to_dicomseg(sorted_base_path = sorted_base_path,
                                                  processed_base_path = processed_base_path,
                                                  dicomseg_json_path = dicomseg_json_path,
                                                  pat_id = pat_id)

---

In [ ]:
from pyplastimatch.utils import viz as viz_utils

In [ ]:
processed_nifti_path = os.path.join(processed_base_path, "nii")

ct_nifti_path = os.path.join(processed_nifti_path, pat_id, pat_id + "_CT.nii.gz")
platipy_output_folder = os.path.join(processed_nifti_path, pat_id, "platipy")

"""
alternative way of loading the resulting NIfTI files
nibabel can sometimes take better care of the orientation of the
converted/segmented images, but will orient the data differently by default
"""

#ct_nii = nib.load(ct_nii_path).dataobj
#seg_nii = nib.load(seg_nii_path).dataobj

ct_nii = sitk.GetArrayFromImage(sitk.ReadImage(ct_nifti_path))

pred_struct_list = [struct for struct in sorted(os.listdir(platipy_output_folder)) if "CN_" not in struct]
pred_struct_names_list = [struct.split(".nii")[0] for struct in pred_struct_list]

pred_segmasks_nifti_list = [os.path.join(platipy_output_folder, struct) for struct in pred_struct_list]

segmask_dict = dict()
segmask_cmap_dict = dict()

for path_to_segmask, segmask in zip(pred_segmasks_nifti_list, pred_struct_names_list):
  segmask_dict[segmask] = sitk.GetArrayFromImage(sitk.ReadImage(path_to_segmask))
  segmask_cmap_dict[segmask] = random.sample([my_reds, my_greens,
                                              my_spring, my_blues], k = 1)[0]

In the next cell, we can visualise the result using a simple widget.

In [ ]:
_ = viz_utils.AxialSliceSegmaskViz(ct_volume = ct_nii,
                                   segmask_dict = segmask_dict,
                                   segmask_cmap_dict = segmask_cmap_dict,
                                   dpi = 100, figsize = (8, 8),
                                   min_hu = -1024, max_hu = 1024)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

---

## **Data Download**

In [ ]:
archive_fn = "%s.zip"%(pat_id)

try:
  os.remove(archive_fn)
except OSError:
  pass

seg_dicom_path = os.path.join(processed_dicomseg_path, pat_id, dicomseg_fn)

!zip -j $archive_fn $ct_nifti_path $seg_dicom_path

  adding: LUNG1-034_CT.nii.gz (deflated 1%)
  adding: LUNG1-034_SEG.dcm (deflated 99%)


In [ ]:
files.download(archive_fn)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>